# 🛠️ QEPC Environment Setup & Validation

**Purpose:** Verify your QEPC environment is correctly configured.

**Run this when:**
- 🆕 First time opening the project
- 🔄 After pulling updates from Git
- 🌐 Switching environments (local ↔ cloud)
- 🐛 Troubleshooting import errors

**What it checks:**
1. ✅ Project paths and context
2. ✅ Required data files exist
3. ✅ QEPC modules import correctly
4. ✅ System diagnostics (optional)

---

## 🧩 1. Load QEPC Context & Set Project Root

In [1]:
import os
import sys
from pathlib import Path

print("🔍 Locating QEPC project root...\n")

# Try direct import first
try:
    from notebook_context import *
    print("✅ Imported notebook_context directly")
    
except ModuleNotFoundError:
    print("ℹ️  notebook_context not on path, searching...")
    
    # Search current directory and parents
    cwd = Path.cwd()
    candidates = [cwd, cwd.parent, cwd.parent.parent]
    
    found_root = None
    for root in candidates:
        if (root / "notebook_context.py").exists():
            found_root = root
            print(f"   Found at: {root}")
            break
    
    if found_root is None:
        raise FileNotFoundError(
            f"❌ Could not find notebook_context.py\n"
            f"   Searched: {cwd} and parent directories\n"
            f"   Ensure you're in the qepc_project folder"
        )
    
    # Add to path and re-import
    sys.path.insert(0, str(found_root))
    os.chdir(found_root)
    
    from notebook_context import *
    print("✅ Imported after path adjustment")

# Verify project_root is defined
try:
    project_root
except NameError:
    project_root = Path.cwd()
    print("⚠️  project_root not defined, using CWD")

print(f"\n📁 Project Root: {project_root}")
print(f"📂 Working Dir:  {os.getcwd()}")
print("\n" + "="*60)

🔍 Locating QEPC project root...

ℹ️  notebook_context not on path, searching...
   Found at: C:\Users\wdorsey\qepc_project
[QEPC Paths] Project Root set: C:\Users\wdorsey\qepc_project
[QEPC] Autoload complete.
[QEPC] Root Shim Restored. Forwarding to qepc.autoload...
✅ Imported after path adjustment
⚠️  project_root not defined, using CWD

📁 Project Root: C:\Users\wdorsey\qepc_project
📂 Working Dir:  C:\Users\wdorsey\qepc_project



---

## 📂 2. Verify Project Structure

In [2]:
print("🔍 Checking project structure...\n")

# Required files and folders
checks = {
    "📄 Schedule": project_root / "data" / "Games.csv",
    "📄 Team Stats": project_root / "data" / "raw" / "Team_Stats.csv",
    "📄 Player Stats": project_root / "data" / "raw" / "PlayerStatistics.csv",
    "📄 Autoloader": project_root / "qepc_autoload.py",
    "📁 QEPC Package": project_root / "qepc",
    "📁 Notebooks": project_root / "notebooks",
    "📁 Data Folder": project_root / "data",
}

# Optional injury file (any version is fine)
injury_files = [
    project_root / "data" / "Injury_Overrides.csv",
    project_root / "data" / "Injury_Overrides_MASTER.csv",
    project_root / "data" / "Injury_Overrides_data_driven.csv",
]

missing = []
found = []

for name, path in checks.items():
    if path.exists():
        print(f"✅ {name}")
        found.append(name)
    else:
        print(f"❌ {name}: {path}")
        missing.append((name, path))

# Check injury files (only need one)
injury_found = any(f.exists() for f in injury_files)
if injury_found:
    found_file = next(f for f in injury_files if f.exists())
    print(f"✅ 📄 Injury Data: {found_file.name}")
else:
    print("⚠️  📄 Injury Data: None found (optional)")

# Summary
print("\n" + "="*60)
if missing:
    print(f"⚠️  {len(missing)} items missing")
    print("\nMissing items:")
    for name, path in missing:
        print(f"   • {name}: {path}")
else:
    print("✅ All required files present!")
print("="*60)

🔍 Checking project structure...

✅ 📄 Schedule
✅ 📄 Team Stats
✅ 📄 Player Stats
✅ 📄 Autoloader
✅ 📁 QEPC Package
✅ 📁 Notebooks
✅ 📁 Data Folder
✅ 📄 Injury Data: Injury_Overrides.csv

✅ All required files present!


---

## 🔬 3. Test QEPC Imports

In [3]:
print("🔍 Testing QEPC module imports...\n")

import_errors = []

def test_import(label, import_func):
    """Test a single import and report status"""
    try:
        import_func()
        print(f"✅ {label}")
        return True
    except Exception as e:
        print(f"❌ {label}: {type(e).__name__}")
        import_errors.append((label, e))
        return False

# Test each major module
test_import("Autoload", lambda: __import__('qepc_autoload'))
test_import("Lambda Engine", lambda: __import__('qepc.core.lambda_engine', fromlist=['compute_lambda']))
test_import("Simulator", lambda: __import__('qepc.core.simulator', fromlist=['run_qepc_simulation']))
test_import("NBA Sim", lambda: __import__('qepc.sports.nba.sim', fromlist=['get_today_games']))
test_import("Backtest Engine", lambda: __import__('qepc.backtest.backtest_engine', fromlist=['run_season_backtest']))

# Summary
print("\n" + "="*60)
if import_errors:
    print(f"⚠️  {len(import_errors)} import(s) failed")
    print("\nFailed imports:")
    for label, error in import_errors:
        print(f"   • {label}: {error}")
    print("\n💡 Tip: Check if the module file exists in qepc/ folder")
else:
    print("✅ All QEPC modules imported successfully!")
print("="*60)

🔍 Testing QEPC module imports...

✅ Autoload
✅ Lambda Engine
✅ Simulator
✅ NBA Sim
✅ Backtest Engine

✅ All QEPC modules imported successfully!


---

## 🧪 4. Run System Diagnostics (Optional)

*This runs a comprehensive check of data integrity, schemas, and system health.*

In [4]:
print("🔍 Running system diagnostics...\n")

try:
    from qepc.utils.diagnostics import run_system_check
    
    # Try with project_root argument
    try:
        result = run_system_check(project_root)
    except TypeError:
        # If function doesn't take arguments, try without
        result = run_system_check()
    
    print("\n✅ Diagnostics completed")
    if result:
        print(f"   Result type: {type(result)}")
    
except ImportError:
    print("ℹ️  Diagnostics module not found (optional feature)")
except Exception as e:
    print(f"⚠️  Diagnostics failed: {type(e).__name__}: {e}")

🔍 Running system diagnostics...

🚀 QEPC SYSTEM DIAGNOSTICS INITIALIZED...

✅ Project Root: OK – Resolved to C:\Users\wdorsey\qepc_project

🔍 Checking required files...
✅ Canonical Schedule: OK – C:\Users\wdorsey\qepc_project\data\Games.csv
✅ Raw Player Stats: OK – C:\Users\wdorsey\qepc_project\data\raw\PlayerStatistics.csv
❌ Raw Team Stats: FAIL – Missing at C:\Users\wdorsey\qepc_project\data\raw\TeamStatistics.csv
✅ Autoload Context: OK – C:\Users\wdorsey\qepc_project\qepc_autoload.py
✅ Restore Guide (Notebook): OK – C:\Users\wdorsey\qepc_project\RESTORE_GUIDE.ipynb
✅ Restore Guide (Markdown): OK – C:\Users\wdorsey\qepc_project\notebooks\RESTORE_GUIDE.md


Check,Status,Details
Canonical Schedule,OK,C:\Users\wdorsey\qepc_project\data\Games.csv
Raw Player Stats,OK,C:\Users\wdorsey\qepc_project\data\raw\PlayerStatistics.csv
Raw Team Stats,FAIL,Missing at C:\Users\wdorsey\qepc_project\data\raw\TeamStatistics.csv
Autoload Context,OK,C:\Users\wdorsey\qepc_project\qepc_autoload.py
Restore Guide (Notebook),OK,C:\Users\wdorsey\qepc_project\RESTORE_GUIDE.ipynb
Restore Guide (Markdown),OK,C:\Users\wdorsey\qepc_project\notebooks\RESTORE_GUIDE.md



📊 Checking data schemas (where files exist)...
✅ Schema: data/Games.csv: OK – All expected columns present.
✅ Schema: data/raw/PlayerStatistics.csv: OK – All expected columns present.


Check,Status,Details
data/Games.csv,OK,All expected columns present.
data/raw/PlayerStatistics.csv,OK,All expected columns present.



🧪 Checking key module imports...
✅ Module: qepc.autoload.paths: OK – Loaded
✅ Module: qepc.core.lambda_engine: OK – Loaded
✅ Module: qepc.core.simulator: OK – Loaded
✅ Module: qepc.sports.nba.sim: OK – Loaded
✅ Module: qepc.sports.nba.strengths_v2: OK – Loaded
✅ Module: qepc.sports.nba.player_data: OK – Loaded
✅ Module: qepc.sports.nba.opponent_data: OK – Loaded
✅ Module: qepc.utils.backup: OK – Loaded
✅ Module: qepc.backtest.backtest_engine: OK – Loaded


Check,Status,Details
qepc.autoload.paths,OK,Loaded successfully
qepc.core.lambda_engine,OK,Loaded successfully
qepc.core.simulator,OK,Loaded successfully
qepc.sports.nba.sim,OK,Loaded successfully
qepc.sports.nba.strengths_v2,OK,Loaded successfully
qepc.sports.nba.player_data,OK,Loaded successfully
qepc.sports.nba.opponent_data,OK,Loaded successfully
qepc.utils.backup,OK,Loaded successfully
qepc.backtest.backtest_engine,OK,Loaded successfully



✨ DIAGNOSTICS COMPLETE.

✅ Diagnostics completed
   Result type: <class 'dict'>


---

## ✅ 5. Environment Status Summary

In [5]:
print("="*60)
print("🎯 QEPC ENVIRONMENT STATUS")
print("="*60)

# Count issues
total_issues = len(missing if 'missing' in dir() else []) + len(import_errors if 'import_errors' in dir() else [])

if total_issues == 0:
    print("\n✅ READY TO GO! Your environment is fully configured.\n")
    print("📚 Next Steps:")
    print("   • 🎯 notebooks/01_core/qepc_dashboard.ipynb - Daily predictions")
    print("   • 📊 notebooks/01_core/qepc_backtest.ipynb - Model validation")
    print("   • 🔧 notebooks/02_utilities/ - Data fetching & management")
    print("   • 🧪 notebooks/03_dev/qepc_sandbox.ipynb - Experimentation\n")
else:
    print(f"\n⚠️  Found {total_issues} issue(s) - review sections above\n")
    if 'missing' in dir() and missing:
        print("Missing files:")
        for name, path in missing:
            print(f"   • {name}")
    if 'import_errors' in dir() and import_errors:
        print("\nImport errors:")
        for label, error in import_errors:
            print(f"   • {label}")
    print("\n💡 Tips:")
    print("   • Ensure you're in the qepc_project directory")
    print("   • Check if files were deleted or moved")
    print("   • Pull latest from Git if working with a team")
    print("   • Run from project root, not a subfolder\n")

print("="*60)

🎯 QEPC ENVIRONMENT STATUS

✅ READY TO GO! Your environment is fully configured.

📚 Next Steps:
   • 🎯 notebooks/01_core/qepc_dashboard.ipynb - Daily predictions
   • 📊 notebooks/01_core/qepc_backtest.ipynb - Model validation
   • 🔧 notebooks/02_utilities/ - Data fetching & management
   • 🧪 notebooks/03_dev/qepc_sandbox.ipynb - Experimentation

